In [326]:
import os

import pandas as pd
import numpy as np

data_dir = 'D:/Projects/sandbox/data/tz_opendata_ua/'
sample = f'{data_dir}tz_opendata_z01012021_po01072021.csv'
workdir = 'D:/Projects/sandbox/etl_pandas/'

In [327]:
dtypes_mapper = {
    'person': 'category',
    'fuel': 'category',
    'kind': 'category',
    'purpose': 'category',
    'reg_addr_koatuu': 'Int64',
    'oper_code': 'Int64',
    'dep_code': 'Int64',
    'make_year': 'Int64',
    'capacity': 'Int64',
    'own_weight': 'Int64',
    'total_weight': 'Int64'
}
na_values_mapper = {
    'reg_addr_koatuu': 0
}

In [328]:
def split_to_entities(df, cols):
    df = (
        df[cols]
        .drop_duplicates()
        .reset_index()
        .drop('index', axis=1)
        .rename(mapper={x: x.lower() for x in cols}, axis=1)
    )
    return df

In [329]:
def add_entities_id(df, name):
    df[f'{name}_id'] = df.index
    df.__setattr__('name', name)
    return df

2070457678

In [ ]:
def replace_values_with_entity_id(df, entity):
    

In [334]:
# dataset example
df_header = pd.read_csv(
    sample,
    sep=';',
    nrows=0
).rename(columns=str.lower)

df = pd.read_csv(
    sample,
    sep=';',
    nrows=10000,
    on_bad_lines='skip',
    dtype=dtypes_mapper,
    na_values=na_values_mapper,
    header=0,
    names=list(df_header),         
    parse_dates=['d_reg']
)
df.head()

,person,reg_addr_koatuu,oper_code,oper_name,d_reg,dep_code,dep,brand,model,vin,make_year,color,kind,body,purpose,fuel,capacity,own_weight,total_weight,n_reg_new
0,P,6310136600,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-02-04,12355,ТСЦ 6341,PEUGEOT,3008,VF30U9HR8BS338559,2011,ЧОРНИЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,1560,1423,2030,АХ5957КЕ
1,P,8036100000,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-06-04,13960,ТСЦ 8048,TOYOTA,COROLLA,JTNBV56E70J182361,2012,СІРИЙ,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,БЕНЗИН,1598,1250,1760,КА5612СА
2,P,3510100000,310,ПЕРЕРЕЄСТРАЦІЯ НА НОВОГО ВЛАСНИКА ЗА ДОГ. КУП....,2021-02-19,12290,ТСЦ 8041,LAND ROVER,DISCOVERY,SALLAAAF4CA632784,2012,СИНІЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,2993,2643,3240,ВА2719СО
3,P,<NA>,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-10-04,13960,ТСЦ 8048,RENAULT,KOLEOS,VF1VY0C0VUC394069,2011,СІРИЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,БЕНЗИН,2488,1750,2250,КА4200СІ
4,P,<NA>,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-05-18,12381,ТСЦ 7145,NISSAN,NOTE,SJNFAAE11U2161945,2012,СИНІЙ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,БЕНЗИН,1386,1164,1546,СА4275ІС


In [341]:
df_veh = split_to_entities(df, vehicle_cols)

In [343]:
df_veh.__setattr__('name', 'vehicle')

In [352]:
df_veh[f'{df_veh.name}_id'] = str(hash(df_veh.name)) + '.' + pd.Series(df_veh.index).astype('str')

In [353]:
df_veh

,brand,model,capacity,fuel,kind,body,purpose,own_weight,total_weight,vehicle_id
0,PEUGEOT,3008,1560,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030,1258530869.0
1,TOYOTA,COROLLA,1598,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760,1258530869.1
2,LAND ROVER,DISCOVERY,2993,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240,1258530869.2
3,RENAULT,KOLEOS,2488,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250,1258530869.3
4,NISSAN,NOTE,1386,БЕНЗИН,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546,1258530869.4
...,...,...,...,...,...,...,...,...,...,...
8578,RENDERS,B 30,<NA>,NaN,НАПІВПРИЧІП,Н/ПР-ПЛАТФОРМА,СПЕЦІАЛІЗОВАНИЙ,7100,39000,1258530869.8578
8579,TOYOTA,CAMRY,2494,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1400,1940,1258530869.8579
8580,HYUNDAI,I10,1248,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1000,1430,1258530869.8580
8581,OPEL,VECTRA,1798,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1475,2030,1258530869.8581


In [354]:
df_reg_cars = split_to_entities(df, cars_for_registration_cols)

In [357]:
df_reg_cars.merge(df_veh, how='left', on=vehicle_cols).drop(vehicle_cols, axis=1)

,vin,make_year,color,vehicle_id
0,VF30U9HR8BS338559,2011,ЧОРНИЙ,1258530869.0
1,JTNBV56E70J182361,2012,СІРИЙ,1258530869.1
2,SALLAAAF4CA632784,2012,СИНІЙ,1258530869.2
3,VF1VY0C0VUC394069,2011,СІРИЙ,1258530869.3
4,SJNFAAE11U2161945,2012,СИНІЙ,1258530869.4
...,...,...,...,...
9646,YA5B302A839R52633,1998,СИНІЙ,1258530869.8578
9647,JTNBF4FK303022320,2013,ЧОРНИЙ,1258530869.8579
9648,MALAM51CP9M220392,2008,СІРИЙ,1258530869.8580
9649,Y6D0ZCF3571022319,2007,СІРИЙ,1258530869.8581


In [331]:
# sample splitted to entities by column names
# vehicle table columns
manufacturers_cols = ['brand']
model_cols = ['model']
fuel_cols = ['fuel']
kind_cols = ['kind']
bodytype_cols = ['body']
purpose_cols = ['purpose']
engine_cols = ['capacity', *fuel_cols]

# operations table columns
dept_code_cols = ['dep_code', 'dep'] # employees
dept_cols = ['dep', 'reg_addr_koatuu'] # department
operations_cols = ['oper_code', 'oper_name', *dept_code_cols]
owner_type_cols = ['person']

# registration activities tables 
vehicle_cols = [*manufacturers_cols, *model_cols, *engine_cols, *kind_cols, *bodytype_cols, *purpose_cols, 'own_weight', 'total_weight']
cars_for_registration_cols = [*vehicle_cols, 'vin', 'make_year', 'color']
license_plates_cols = ['vin', 'n_reg_new']
owners_cols = ['person']

# table registration_activities based on smaller entities which described earlier
registration_activities = [*operations_cols, *cars_for_registration_cols, *owners_cols, *license_plates_cols]

In [359]:
df_reg_activities = split_to_entities(df, registration_activities)

In [364]:
cars_for_registration_cols

['brand',
 'model',
 'capacity',
 'fuel',
 'kind',
 'body',
 'purpose',
 'own_weight',
 'total_weight',
 'vin',
 'make_year',
 'color']

In [363]:
df_reg_cars

,brand,model,capacity,fuel,kind,body,purpose,own_weight,total_weight,vin,make_year,color
0,PEUGEOT,3008,1560,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030,VF30U9HR8BS338559,2011,ЧОРНИЙ
1,TOYOTA,COROLLA,1598,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760,JTNBV56E70J182361,2012,СІРИЙ
2,LAND ROVER,DISCOVERY,2993,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240,SALLAAAF4CA632784,2012,СИНІЙ
3,RENAULT,KOLEOS,2488,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250,VF1VY0C0VUC394069,2011,СІРИЙ
4,NISSAN,NOTE,1386,БЕНЗИН,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546,SJNFAAE11U2161945,2012,СИНІЙ
...,...,...,...,...,...,...,...,...,...,...,...,...
9646,RENDERS,B 30,<NA>,NaN,НАПІВПРИЧІП,Н/ПР-ПЛАТФОРМА,СПЕЦІАЛІЗОВАНИЙ,7100,39000,YA5B302A839R52633,1998,СИНІЙ
9647,TOYOTA,CAMRY,2494,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1400,1940,JTNBF4FK303022320,2013,ЧОРНИЙ
9648,HYUNDAI,I10,1248,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1000,1430,MALAM51CP9M220392,2008,СІРИЙ
9649,OPEL,VECTRA,1798,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1475,2030,Y6D0ZCF3571022319,2007,СІРИЙ


In [360]:
df_reg_activities

,oper_code,oper_name,dep_code,dep,brand,model,capacity,fuel,kind,body,purpose,own_weight,total_weight,vin,make_year,color,person,vin,n_reg_new
0,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,12355,ТСЦ 6341,PEUGEOT,3008,1560,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030,VF30U9HR8BS338559,2011,ЧОРНИЙ,P,VF30U9HR8BS338559,АХ5957КЕ
1,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,13960,ТСЦ 8048,TOYOTA,COROLLA,1598,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760,JTNBV56E70J182361,2012,СІРИЙ,P,JTNBV56E70J182361,КА5612СА
2,310,ПЕРЕРЕЄСТРАЦІЯ НА НОВОГО ВЛАСНИКА ЗА ДОГ. КУП....,12290,ТСЦ 8041,LAND ROVER,DISCOVERY,2993,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240,SALLAAAF4CA632784,2012,СИНІЙ,P,SALLAAAF4CA632784,ВА2719СО
3,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,13960,ТСЦ 8048,RENAULT,KOLEOS,2488,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250,VF1VY0C0VUC394069,2011,СІРИЙ,P,VF1VY0C0VUC394069,КА4200СІ
4,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,12381,ТСЦ 7145,NISSAN,NOTE,1386,БЕНЗИН,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546,SJNFAAE11U2161945,2012,СИНІЙ,P,SJNFAAE11U2161945,СА4275ІС
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,308,ПЕРЕРЕЄСТРАЦІЯ НА НОВОГО ВЛАСНИКА ЗА ДОГ. КУПI...,12373,ТСЦ 6843,RENDERS,B 30,<NA>,NaN,НАПІВПРИЧІП,Н/ПР-ПЛАТФОРМА,СПЕЦІАЛІЗОВАНИЙ,7100,39000,YA5B302A839R52633,1998,СИНІЙ,P,YA5B302A839R52633,ВХ1052ХF
9991,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,12295,ТСЦ 8046,TOYOTA,CAMRY,2494,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1400,1940,JTNBF4FK303022320,2013,ЧОРНИЙ,P,JTNBF4FK303022320,АІ0040ВЕ
9992,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,12356,ТСЦ 6342,HYUNDAI,I10,1248,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1000,1430,MALAM51CP9M220392,2008,СІРИЙ,P,MALAM51CP9M220392,АХ5626КІ
9993,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,12330,ТСЦ 5341,OPEL,VECTRA,1798,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1475,2030,Y6D0ZCF3571022319,2007,СІРИЙ,J,Y6D0ZCF3571022319,ВІ5604НВ


In [122]:
%%time
df_manf = split_to_entities(df, manufacturers_cols)
df_models = split_to_entities(df, model_cols)
df_fuel = split_to_entities(df, fuel_cols)
df_kind = split_to_entities(df, kind_cols)
df_bodytype = split_to_entities(df, bodytype_cols)
df_purpose = split_to_entities(df, purpose_cols)
df_engine = split_to_entities(df, engine_cols)
df_dept_code = split_to_entities(df, dept_code_cols)
df_dept = split_to_entities(df, dept_cols)
df_operations = split_to_entities(df, operations_cols)
df_owner = split_to_entities(df, owner_type_cols)
df_license_plates = split_to_entities(df, license_plates_cols)
df_vehicle = split_to_entities(df, vehicle_cols)
df_cars_for_registration = split_to_entities(df, cars_for_registration_cols)
df_registration_activities = split_to_entities(df, registration_activities)

Wall time: 119 ms


In [176]:
header = pd.read_csv(
    sample,
    sep=';',
    nrows=0
).rename(columns=str.lower)

Index(['person', 'reg_addr_koatuu', 'oper_code', 'oper_name', 'd_reg',
       'dep_code', 'dep', 'brand', 'model', 'vin', 'make_year', 'color',
       'kind', 'body', 'purpose', 'fuel', 'capacity', 'own_weight',
       'total_weight', 'n_reg_new'],
      dtype='object')

In [193]:
pd.read_csv(sample,
    sep=';',
    nrows=100,
    on_bad_lines='skip',
    dtype=dtypes_mapper,
    na_values=na_values_mapper,
    header=0,
    names=list(header),
    parse_dates=['d_reg']
)

,person,reg_addr_koatuu,oper_code,oper_name,d_reg,dep_code,dep,brand,model,vin,make_year,color,kind,body,purpose,fuel,capacity,own_weight,total_weight,n_reg_new
0,P,6310136600,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-02-04,12355,ТСЦ 6341,PEUGEOT,3008,VF30U9HR8BS338559,2011,ЧОРНИЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,1560,1423,2030,АХ5957КЕ
1,P,8036100000,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-06-04,13960,ТСЦ 8048,TOYOTA,COROLLA,JTNBV56E70J182361,2012,СІРИЙ,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,БЕНЗИН,1598,1250,1760,КА5612СА
2,P,3510100000,310,ПЕРЕРЕЄСТРАЦІЯ НА НОВОГО ВЛАСНИКА ЗА ДОГ. КУП....,2021-02-19,12290,ТСЦ 8041,LAND ROVER,DISCOVERY,SALLAAAF4CA632784,2012,СИНІЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,2993,2643,3240,ВА2719СО
3,P,<NA>,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-10-04,13960,ТСЦ 8048,RENAULT,KOLEOS,VF1VY0C0VUC394069,2011,СІРИЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,БЕНЗИН,2488,1750,2250,КА4200СІ
4,P,<NA>,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-05-18,12381,ТСЦ 7145,NISSAN,NOTE,SJNFAAE11U2161945,2012,СИНІЙ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,БЕНЗИН,1386,1164,1546,СА4275ІС
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,P,5322486301,284,НАЛЕЖНИЙ КОРИСТУВАЧ. ВИКЛЮЧЕННЯ ВІДОМОСТЕЙ,2021-08-05,12331,ТСЦ 5342,HYUNDAI,I10,MALAN51CBCM153941,2012,БІЛИЙ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,БЕНЗИН,1248,1000,1430,ВІ8043ЕВ
96,P,2323081801,250,ВИДАЧА ТИМЧАСОВОГО РЕЄСТР ТАЛОНУ ПО ДОРУЧЕННЮ ...,2021-05-22,12269,ТСЦ 2343,CHEVROLET,LACETTI,KL1NF1961CK713652,2012,СИНІЙ,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,БЕНЗИН АБО ГАЗ,1598,1297,1660,АР1230ЕХ
97,P,<NA>,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-06-17,12351,ТСЦ 6143,MERCEDES-BENZ,GL 350 CDI,WDC1648221A654282,2010,ЧОРНИЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,2987,2450,3250,ВО6456СТ
98,P,8038900000,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-03-18,12375,ТСЦ 6845,KIA,RIO,Z94CB41ABCR042235,2012,СІРИЙ,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,БЕНЗИН,1396,1115,1565,КА2150ВС


In [207]:
df_engine = df_engine.rename(mapper={'id': 'engine_id'}, axis=1)

In [212]:
df_models = df_models.rename(mapper={'id': 'model_id'}, axis=1)

In [216]:
df_manf = df_manf.rename(mapper={'id': 'manufacturer_id'}, axis=1)

In [225]:
df_kind = add_id_to_entities(df_kind, 'kind')
df_kind = df_kind.rename(mapper={'id': 'kind_id'}, axis=1)

In [224]:
df_bodytype = add_id_to_entities(df_bodytype, 'bodytype')
df_bodytype = df_bodytype.rename(mapper={'id': 'bodytype_id'}, axis=1)

In [223]:
df_purpose = add_id_to_entities(df_purpose, 'purpose')
df_purpose = df_purpose.rename(mapper={'id': 'purpose_id'}, axis=1)

In [255]:
df_veh_fully_mapped['vehicle_id'] = df_veh_fully_mapped.index

In [257]:
df_veh_fully_mapped

,own_weight,total_weight,engine_id,model_id,manufacturer_id,kind_id,bodytype_id,purpose_id,vehicle_id
0,1423,2030,0,0,0,0,0,0,0
1,1250,1760,1,1,1,0,1,0,1
2,2643,3240,2,2,2,0,0,0,2
3,1750,2250,3,3,3,0,0,0,3
4,1164,1546,4,4,4,0,2,0,4
...,...,...,...,...,...,...,...,...,...
8578,7100,39000,32,1661,181,2,31,1,8578
8579,1400,1940,252,22,1,0,1,0,8579
8580,1000,1430,147,73,18,0,2,0,8580
8581,1475,2030,89,17,10,0,0,0,8581


In [262]:
df_veh_fully_mapped

,own_weight,total_weight,engine_id,model_id,manufacturer_id,kind_id,bodytype_id,purpose_id,vehicle_id
0,1423,2030,0,0,0,0,0,0,0
1,1250,1760,1,1,1,0,1,0,1
2,2643,3240,2,2,2,0,0,0,2
3,1750,2250,3,3,3,0,0,0,3
4,1164,1546,4,4,4,0,2,0,4
...,...,...,...,...,...,...,...,...,...
8578,7100,39000,32,1661,181,2,31,1,8578
8579,1400,1940,252,22,1,0,1,0,8579
8580,1000,1430,147,73,18,0,2,0,8580
8581,1475,2030,89,17,10,0,0,0,8581


In [266]:
vehicle_cols[0] = 'manufacturer'

In [270]:
df_cars_for_registration = df_cars_for_registration.rename(mapper={'brand':'manufacturer'}, axis=1)

In [ ]:
df

In [286]:
df_veh_fully_mapped.set_index('vehicle_id').columns = [x.replace('_id', '') for x in df_veh_fully_mapped.set_index('vehicle_id').columns]

In [292]:
ls = [x.replace('_id', '') for x in df_veh_fully_mapped.columns if x != 'vehicle_id']

In [293]:
ls.append('vehicle_id')

In [295]:
df_veh_fully_mapped.columns = ls

In [296]:
df_veh_fully_mapped

,own_weight,total_weight,engine,model,manufacturer,kind,bodytype,purpose,vehicle_id
0,1423,2030,0,0,0,0,0,0,0
1,1250,1760,1,1,1,0,1,0,1
2,2643,3240,2,2,2,0,0,0,2
3,1750,2250,3,3,3,0,0,0,3
4,1164,1546,4,4,4,0,2,0,4
...,...,...,...,...,...,...,...,...,...
8578,7100,39000,32,1661,181,2,31,1,8578
8579,1400,1940,252,22,1,0,1,0,8579
8580,1000,1430,147,73,18,0,2,0,8580
8581,1475,2030,89,17,10,0,0,0,8581


In [302]:
df_cars_for_registration[vehicle_cols]

,manufacturer,model,capacity,fuel,kind,body,purpose,own_weight,total_weight
0,PEUGEOT,3008,1560,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030
1,TOYOTA,COROLLA,1598,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760
2,LAND ROVER,DISCOVERY,2993,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240
3,RENAULT,KOLEOS,2488,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250
4,NISSAN,NOTE,1386,БЕНЗИН,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546
...,...,...,...,...,...,...,...,...,...
9646,RENDERS,B 30,<NA>,NaN,НАПІВПРИЧІП,Н/ПР-ПЛАТФОРМА,СПЕЦІАЛІЗОВАНИЙ,7100,39000
9647,TOYOTA,CAMRY,2494,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1400,1940
9648,HYUNDAI,I10,1248,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1000,1430
9649,OPEL,VECTRA,1798,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1475,2030


In [309]:
vehicle_cols

['manufacturer',
 'model',
 'fuel',
 'kind',
 'body',
 'purpose',
 'own_weight',
 'total_weight']

In [319]:
df_cars_for_registration_2 = df_cars_for_registration.merge(df_engine, how='left', on=engine_cols).drop(engine_cols, axis=1)

In [318]:
df_cars_for_registration.merge(df_engine, how='left', on=engine_cols).drop(engine_cols, axis=1)

,manufacturer,model,kind,body,purpose,own_weight,total_weight,vin,make_year,color,engine_id
0,PEUGEOT,3008,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030,VF30U9HR8BS338559,2011,ЧОРНИЙ,0
1,TOYOTA,COROLLA,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760,JTNBV56E70J182361,2012,СІРИЙ,1
2,LAND ROVER,DISCOVERY,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240,SALLAAAF4CA632784,2012,СИНІЙ,2
3,RENAULT,KOLEOS,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250,VF1VY0C0VUC394069,2011,СІРИЙ,3
4,NISSAN,NOTE,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546,SJNFAAE11U2161945,2012,СИНІЙ,4
...,...,...,...,...,...,...,...,...,...,...,...
9646,RENDERS,B 30,НАПІВПРИЧІП,Н/ПР-ПЛАТФОРМА,СПЕЦІАЛІЗОВАНИЙ,7100,39000,YA5B302A839R52633,1998,СИНІЙ,32
9647,TOYOTA,CAMRY,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1400,1940,JTNBF4FK303022320,2013,ЧОРНИЙ,252
9648,HYUNDAI,I10,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1000,1430,MALAM51CP9M220392,2008,СІРИЙ,147
9649,OPEL,VECTRA,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1475,2030,Y6D0ZCF3571022319,2007,СІРИЙ,89


In [321]:
df_veh_fully_mapped

,own_weight,total_weight,engine,model,manufacturer,kind,bodytype,purpose,vehicle_id
0,1423,2030,0,0,0,0,0,0,0
1,1250,1760,1,1,1,0,1,0,1
2,2643,3240,2,2,2,0,0,0,2
3,1750,2250,3,3,3,0,0,0,3
4,1164,1546,4,4,4,0,2,0,4
...,...,...,...,...,...,...,...,...,...
8578,7100,39000,32,1661,181,2,31,1,8578
8579,1400,1940,252,22,1,0,1,0,8579
8580,1000,1430,147,73,18,0,2,0,8580
8581,1475,2030,89,17,10,0,0,0,8581


In [325]:
df_vehicle

,brand,model,capacity,fuel,kind,body,purpose,own_weight,total_weight
0,PEUGEOT,3008,1560,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030
1,TOYOTA,COROLLA,1598,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760
2,LAND ROVER,DISCOVERY,2993,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240
3,RENAULT,KOLEOS,2488,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250
4,NISSAN,NOTE,1386,БЕНЗИН,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546
...,...,...,...,...,...,...,...,...,...
8578,RENDERS,B 30,<NA>,NaN,НАПІВПРИЧІП,Н/ПР-ПЛАТФОРМА,СПЕЦІАЛІЗОВАНИЙ,7100,39000
8579,TOYOTA,CAMRY,2494,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1400,1940
8580,HYUNDAI,I10,1248,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1000,1430
8581,OPEL,VECTRA,1798,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1475,2030


In [324]:
df_cars_for_registration_2

,manufacturer,model,kind,body,purpose,own_weight,total_weight,vin,make_year,color,engine_id
0,PEUGEOT,3008,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030,VF30U9HR8BS338559,2011,ЧОРНИЙ,0
1,TOYOTA,COROLLA,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760,JTNBV56E70J182361,2012,СІРИЙ,1
2,LAND ROVER,DISCOVERY,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240,SALLAAAF4CA632784,2012,СИНІЙ,2
3,RENAULT,KOLEOS,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250,VF1VY0C0VUC394069,2011,СІРИЙ,3
4,NISSAN,NOTE,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546,SJNFAAE11U2161945,2012,СИНІЙ,4
...,...,...,...,...,...,...,...,...,...,...,...
9646,RENDERS,B 30,НАПІВПРИЧІП,Н/ПР-ПЛАТФОРМА,СПЕЦІАЛІЗОВАНИЙ,7100,39000,YA5B302A839R52633,1998,СИНІЙ,32
9647,TOYOTA,CAMRY,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1400,1940,JTNBF4FK303022320,2013,ЧОРНИЙ,252
9648,HYUNDAI,I10,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1000,1430,MALAM51CP9M220392,2008,СІРИЙ,147
9649,OPEL,VECTRA,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1475,2030,Y6D0ZCF3571022319,2007,СІРИЙ,89


In [322]:
df_cars_for_registration_2

,manufacturer,model,kind,body,purpose,own_weight,total_weight,vin,make_year,color,engine_id
0,PEUGEOT,3008,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030,VF30U9HR8BS338559,2011,ЧОРНИЙ,0
1,TOYOTA,COROLLA,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760,JTNBV56E70J182361,2012,СІРИЙ,1
2,LAND ROVER,DISCOVERY,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240,SALLAAAF4CA632784,2012,СИНІЙ,2
3,RENAULT,KOLEOS,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250,VF1VY0C0VUC394069,2011,СІРИЙ,3
4,NISSAN,NOTE,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546,SJNFAAE11U2161945,2012,СИНІЙ,4
...,...,...,...,...,...,...,...,...,...,...,...
9646,RENDERS,B 30,НАПІВПРИЧІП,Н/ПР-ПЛАТФОРМА,СПЕЦІАЛІЗОВАНИЙ,7100,39000,YA5B302A839R52633,1998,СИНІЙ,32
9647,TOYOTA,CAMRY,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1400,1940,JTNBF4FK303022320,2013,ЧОРНИЙ,252
9648,HYUNDAI,I10,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1000,1430,MALAM51CP9M220392,2008,СІРИЙ,147
9649,OPEL,VECTRA,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1475,2030,Y6D0ZCF3571022319,2007,СІРИЙ,89


In [310]:
df_cars_for_registration.merge(df_veh_fully_mapped, how='left', on=vehicle_cols).drop(vehicle_cols, axis=1)

KeyError: 'fuel'

In [228]:
df_veh_fully_mapped = (
    df_vehicle
    .merge(df_engine, how='left', on=engine_cols).drop(engine_cols, axis=1)
    .merge(df_models, how='left', on=model_cols).drop(model_cols, axis=1)
    .merge(df_manf, how='left', on=manufacturers_cols).drop(manufacturers_cols, axis=1)
    .merge(df_kind, how='left', on=kind_cols).drop(kind_cols, axis=1)
    .merge(df_bodytype, how='left', on=bodytype_cols).drop(bodytype_cols, axis=1)
    .merge(df_purpose, how='left', on=purpose_cols).drop(purpose_cols, axis=1)
)

In [231]:
df_vehicle.memory_usage()

Index              64
brand           34332
model           34332
capacity        77247
fuel             8775
kind             8771
body            34332
purpose          8659
own_weight      77247
total_weight    77247
dtype: int64

In [239]:
print(df_veh_fully_mapped['engine_id'].memory_usage())
print(df_veh_fully_mapped['engine_id'].astype('int64').memory_usage())
print(df_veh_fully_mapped['engine_id'].astype('int32').memory_usage())
print(df_veh_fully_mapped['engine_id'].astype('int16').memory_usage())

137328
137328
102996
85830


In [250]:
df_veh_fully_mapped['engine_id'].astype('int32') == df_veh_fully_mapped['engine_id'].astype('int64')

0       True
1       True
2       True
3       True
4       True
        ... 
8578    True
8579    True
8580    True
8581    True
8582    True
Name: engine_id, Length: 8583, dtype: bool

In [145]:
def add_id_to_entities(df, name):
    df['id'] = df.index
    df.__setattr__('name', name)
    return df

In [147]:
df_models = add_id_to_entities(df_models, 'models')
df_models.name

'models'

In [148]:
df_fuel = add_id_to_entities(df_fuel, 'fuel')
df_engine = add_id_to_entities(df_engine, 'engine')

In [160]:
df_engine_mapped = df_engine.merge(df_fuel, how='left', left_on='fuel', right_on='fuel').drop('fuel', axis=1).rename(mapper={'id_y': 'fuel_id', 'id_x': 'engine_id'}, axis=1)

In [164]:
df_engine

,capacity,fuel,id
0,1560,ДИЗЕЛЬНЕ ПАЛИВО,0
1,1598,БЕНЗИН,1
2,2993,ДИЗЕЛЬНЕ ПАЛИВО,2
3,2488,БЕНЗИН,3
4,1386,БЕНЗИН,4
...,...,...,...
911,3800,ДИЗЕЛЬНЕ ПАЛИВО,911
912,1203,БЕНЗИН,912
913,2706,БЕНЗИН,913
914,9000,ДИЗЕЛЬНЕ ПАЛИВО,914


In [150]:
df_fuel

,fuel,id
0,ДИЗЕЛЬНЕ ПАЛИВО,0
1,БЕНЗИН,1
2,БЕНЗИН АБО ГАЗ,2
3,NaN,3
4,ЕЛЕКТРО АБО БЕНЗИН,4
5,"БЕНЗИН, ГАЗ АБО ЕЛЕКТРО",5
6,ВІДСУТНЄ,6
7,НЕ ВИЗНАЧЕНО,7
8,ДИЗЕЛЬНЕ ПАЛИВО АБО ГАЗ,8


In [111]:
import uuid

In [128]:
df_vehicle.set_index('brand').join(df_manf.reset_index()
                                   .set_index('brand'))

,model,capacity,fuel,kind,body,purpose,own_weight,total_weight,index
brand,,,,,,,,,
ACKERMANN,FRUE,<NA>,NaN,ПРИЧІП,ПР-ФУРГОН,СПЕЦІАЛІЗОВАНИЙ,3450,6250,207
ACURA,RDX,3471,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1686,2286,78
ACURA,MDX,3664,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2070,2700,78
ACURA,RDX,2300,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1550,2100,78
ACURA,MDX,3664,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1720,2040,78
...,...,...,...,...,...,...,...,...,...
УРАЛ,4320,11150,ДИЗЕЛЬНЕ ПАЛИВО,ВАНТАЖНИЙ,САМОСКИД,СПЕЦІАЛІЗОВАНИЙ,8020,13245,56
ХАЗ,3250.02,3972,ДИЗЕЛЬНЕ ПАЛИВО,АВТОБУС,АВТОБУС,ЗАГАЛЬНИЙ,4360,6800,117
ХАЗ,3230.22,3972,ДИЗЕЛЬНЕ ПАЛИВО,АВТОБУС,АВТОБУС,ЗАГАЛЬНИЙ,4650,6950,117


In [141]:
df_manf.__setattr__('name', 'manufacturers')

In [119]:
df_manf['id'] = uuid.uuid1().hex

In [121]:
dfdf_manf.drop('id', axis=1)

,brand
0,PEUGEOT
1,TOYOTA
2,LAND ROVER
3,RENAULT
4,NISSAN
...,...
221,ELBO
222,КЕМЗ
223,TRAILER
224,КНОТТ


In [109]:
df_vehicle

,brand,model,capacity,fuel,kind,body,purpose,own_weight,total_weight
0,PEUGEOT,3008,1560,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030
1,TOYOTA,COROLLA,1598,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760
2,LAND ROVER,DISCOVERY,2993,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240
3,RENAULT,KOLEOS,2488,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250
4,NISSAN,NOTE,1386,БЕНЗИН,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546
...,...,...,...,...,...,...,...,...,...
8578,RENDERS,B 30,<NA>,NaN,НАПІВПРИЧІП,Н/ПР-ПЛАТФОРМА,СПЕЦІАЛІЗОВАНИЙ,7100,39000
8579,TOYOTA,CAMRY,2494,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1400,1940
8580,HYUNDAI,I10,1248,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1000,1430
8581,OPEL,VECTRA,1798,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1475,2030


In [98]:
df_vehicle[['brand', 'model']].drop_duplicates().groupby('brand')[['model']].count().sort_values('model', ascending=False)

,model
brand,
MERCEDES-BENZ,227
ВАЗ,85
BMW,69
ГАЗ,63
MAN,62
...,...
LINCOLN,1
LOHEAC,1
MACK,1


In [ ]:
df_models.sort_values()

In [66]:
split_to_entities(df, operations_cols)

,oper_code,oper_name,dep_code,dep
0,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,12355,ТСЦ 6341
1,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,13960,ТСЦ 8048
2,310,ПЕРЕРЕЄСТРАЦІЯ НА НОВОГО ВЛАСНИКА ЗА ДОГ. КУП....,12290,ТСЦ 8041
3,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,12381,ТСЦ 7145
4,308,ПЕРЕРЕЄСТРАЦІЯ НА НОВОГО ВЛАСНИКА ЗА ДОГ. КУПI...,12307,ТСЦ 4641
...,...,...,...,...
1499,230,ВИДАЧА СВ. ПРО РЕЄСТР ДЛЯ ПОЇЗДКИ ЗА КОРДОН,12272,ТСЦ 2346
1500,560,ЗНЯТТЯ З ОБЛІКУ ПРИ ПОВЕРНЕННІ ІЗ ПОЇЗДКИ ЗА К...,12272,ТСЦ 2346
1501,254,НАЛЕЖНИЙ КОРИСТУВАЧ. РЕЄСТРАЦІЯ,13659,ТСЦ 0546
1502,404,"ПЕРЕРЕЄСТРАЦІЯ ТЗ, В ЗВ’ЯЗКУ ІЗ НАНЕСЕННЯ СПЕЦ...",12233,ТСЦ 0545


In [51]:
def by_cols_reader(cols, sample=sample):
    df = (
        pd.read_csv(sample, sep=';', nrows=10000, on_bad_lines='skip', usecols=cols)
        .drop_duplicates()
        .reset_index()
        .drop('index', axis=1)
        .rename(mapper={x: x.lower() for x in cols}, axis=1)
    )
    return df

In [59]:
df_mnf = by_cols_reader(manufacturers_cols)

In [62]:
%time df_models = by_cols_reader(model_cols)
%time df_fuel = by_cols_reader(fuel_cols)
%time df_engine = by_cols_reader(engine_cols)
%time df_operations = by_cols_reader(operations_cols)

Wall time: 61 ms
Wall time: 50 ms
Wall time: 53 ms
Wall time: 61 ms


In [84]:
import sys

In [85]:
sys.getsizeof(hash('_'.join([str(x) for x in list(df_operations.iloc[0, :].to_numpy())])))

16

In [ ]:
df_operations.iloc[0, :].me

In [86]:
df_operations.iloc[0, :]

651

In [93]:
df_fuel.reset_index().set_index('fuel')

,index
fuel,
ДИЗЕЛЬНЕ ПАЛИВО,0
БЕНЗИН,1
БЕНЗИН АБО ГАЗ,2
NaN,3
ЕЛЕКТРО АБО БЕНЗИН,4
"БЕНЗИН, ГАЗ АБО ЕЛЕКТРО",5
ВІДСУТНЄ,6
НЕ ВИЗНАЧЕНО,7
ДИЗЕЛЬНЕ ПАЛИВО АБО ГАЗ,8


In [98]:
df_engine.set_index('fuel').join(df_fuel.reset_index().set_index('fuel')).rename(mapper={'index': 'fuel_id'}, axis=1).reset_index()

,fuel,capacity,fuel_id
0,БЕНЗИН,1598.0,1
1,БЕНЗИН,2488.0,1
2,БЕНЗИН,1386.0,1
3,БЕНЗИН,1590.0,1
4,БЕНЗИН,1984.0,1
...,...,...,...
911,ЕЛЕКТРО АБО БЕНЗИН,1798.0,4
912,НЕ ВИЗНАЧЕНО,NaN,7
913,NaN,NaN,3
914,NaN,0.0,3


In [11]:
df = pd.read_csv(sample, sep=';', nrows=10000, on_bad_lines='skip')

In [26]:
df_mnf = df[manufacturers_cols]

In [38]:
df_mnf = df_mnf.drop_duplicates().reset_index().drop('index', axis=1)

In [57]:
df_mnf =df_mnf.rename(mapper={x: x.lower() for x in df_mnf.columns}, axis=1)